<table>
    <tr>
        <td><img src='https://coastalrisk.live/wp-content/uploads/2018/05/cera_50x50.png' alt='Image' width='50' height=50'></td>
        <td><h1 align="left"><font color='green'> Introduction to Matplotlib Contouring for ADCIRC NetCDF Data </font></h1></td>
    </tr>
</table>


In [ ]:
# This script uses the python library matplotlib (http://matplotlib.org/) to create contours from a single ADCIRC netcdf file.
# If multiple time steps are given in the ADCIRC netcdf file, the first time step will be extracted.

# Copyright (C): Carola Kaiser 2014-2024, Louisiana State University. 
# This script is part of the Coastal Emergency Risks Assessment (CERA), a real-time visualization system for storm surge guidance.
# See https://cera.coastalrisk.live. This CERA script is Open Source software; distributed under the MIT License.

# Table of Contents

- [Part 1: Software installation and data preparation](#part-1-introduction)
  - [1.1 - Software installation](#11---software-installation)
  - [1.2 - Tutorial example NetCDF file](#12-downloading-the-netcdf-data)
  - [1.3 - Importing the Python libraries](#13---importing-the-python-libraries)

- [Part 2: Data import](#part-2-data-import)
  - [2.1 - Opening a NetCDF file](#21---opening-a-netcdf-file)
  - [2.2 - Extracting and validating grid variables from the NetCDF file](#22---extracting-and-validating-grid-variables-from-netcdf-file)
  - [2.3 - Extracting and validating attribute data from the NetCDF file](#23---extracting-and-validating-attribute-data-from-netcdf-file)

- [Part 3: Map contouring and interactive plotting](#part-3-contouring-and-interactive-plotting)


<br>
Note: If you are running through Google Colab, please use the table of contents from the top left corner.

# Part 1: Introduction
The purpose of this script is to generate contours from ADCIRC NetCDF files using the Matplotlib library. These contours can then be utilized to produce various GIS file formats and map plots, essential for storm surge analysis and emergency management.

The contouring of ADCIRC NetCDF files is one of the center pieces used to create maps for the [CERA](https://cera.coastalrisk.live/) interactive visualization tool. In this Jupyter Notebook tutorial, we'll explore nuances of using Matplotlib's 'tricontourf' function, thereby illuminating the inner workings of CERA, a critical tool for coastal risk assessment.

## 1.1 - Software Installation


### Standalone Python Script

If you prefer to run the tutorial examples using a command-line or terminal, you can use the Python standalone script 'cera_contour_matplotlib.py'. The following libraries should be installed: netCDF4, matplotlib, shapely, numpy.
<hr>

### 1.1.1 Installing Jupyter Notebook

The Jupyter notebook can be installed in several ways. Please choose accordingly to your system requirements and personal preference. 

- [Installing the classic Jupyter Notebook interface](https://docs.jupyter.org/en/latest/install/notebook-classic.html)
- [Installing the JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)



#### Our prefered way of working is using [Viusal Studio Code](https://code.visualstudio.com/download).
The general instruction are [here](https://code.visualstudio.com/docs/datascience/jupyter-notebooks) using the extension provided by Microsoft [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

- Pick the prefered package manager / environment manager (conda or venv).
- Open the folder that contains the jupyter notebook file (ipynb) in vscode.
- From the top right corner select the kernel for this specific jupyter notebook as created in the previous step.

### 1.1.2 Installing the Python libraries

*** Note: This tutorial was tested with Python 3.12, netCDF4 1.6.5, Matplotlib 3.8.4, Shapely 2.0.4, Numpy 1.26.4, and ipywidgets 8.1.2.

- `numpy` is a Python library that provides support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `netCDF4` is a Python interface to the netCDF C library. netCDF (Network Common Data Form) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data.
- `matplotlib` is a plotting library for Python. It provides an object-oriented API for embedding plots into applications using general-purpose GUI toolkits like Tkinter, wxPython, Qt, or GTK. Matplotlib is also a popular library for creating static, animated, and interactive visualizations in Python.
- `shapely` is a Python package for set-theoretic analysis and manipulation of planar features using functions from the GEOS library (which is the engine of PostGIS).
- `ipywidgets` are interactive HTML widgets for Jupyter notebooks and the IPython kernel. Widgets in ipywidgets are objects that represent a control, like a slider, a text box, a button, etc. They can be used to build interactive GUIs for your notebooks, handle user input, and render interactive output. Also, they can also be used to synchronize stateful and stateless information between Python and JavaScript.


In [ ]:
!pip install numpy
!pip install netCDF4
!pip install matplotlib
!pip install shapely
!pip install ipywidgets

## 1.2 - Tutorial example NetCDF file
In our exercise, we will use an example NetCDF file coming from the oceanographic ADCIRC model. The file contains the underlying mesh topology and the values for the maximum water elevation computed at each mesh node. 

If you do not have the 'wget' tool installed, you can downlaod the file by adding the URL in a web browser. The file is also available in the directory of this script. Notebook/maxele_contouring.63.nc

In [ ]:
!wget https://cloud.cera.lsu.edu/s/frcZtNgANgdgCXP/download/maxele_contouring.63.nc

## 1.3 - Importing the Python libraries
When we start a Python script, we need to import the necessary package(s) that the script will use. This tutorial was tested with Python 3.12.

In [ ]:
# importing libraries
import os, sys
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib import path
from shapely.geometry import geo, Polygon, Point, MultiPolygon
from operator import itemgetter
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

# Part 2: Data Import

## 2.1 - Opening a NetCDF file 

```python
myfile = "maxele_contouring.63.nc"
```
The netCDF file name "maxele_contouring.63.nc" is assigned to the variable `myfile`. The netCDF contains the grid information (x and y coordinates and grid elements) and the associated data arrays. The grid information and the data can also be provided as separate netCDFs but are included in one file for the purpose of this tutorial.

```python
vars = netCDF4.Dataset(myfile).variables
```
We use the `netCDF4.Dataset` function to open the netCDF file specified by `myfile`. The `Dataset` function returns a `Dataset` object that represents the netCDF file. The `.variables` attribute of a `Dataset` object is a dictionary where each key-value pair represents a variable from a netCDF dataset. The keys are the variable names, while the values are the corresponding variable objects.

In [ ]:
# Open the NetCDF file
myfile = "maxele_contouring.63.nc"
vars = netCDF4.Dataset(myfile).variables

## 2.2 - Extracting and validating grid variables from the NetCDF file

We can print the assigned `vars` Dataset to see what variables we have in the NetCDF.

`print(vars)` returns a full list of attributes of all variables including long names, units, NoData values etc.

In [ ]:
print(vars)

`print(vars.keys())` is printing the names of all the variables in the netCDF dataset. This can be useful for a quick understanding what data is available in the dataset, for debugging, or for logging.

In [ ]:
print(vars.keys())

Let's read the grid variables 'x', 'y', and 'element' (or 'lon', 'lat', and 'ele' if the former are not present) data from the netCDF stored in `vars`, adjusting the element indexing from 1-based to 0-based. 

We can also check if any of these data arrays are `None`, and if so, print an error message and exit the program.

In [ ]:
# Get variable names for x, y depending on grid
if 'x' in vars:
    var_x = 'x'
    var_y = 'y'
    var_element = 'element'
else:
    var_x = 'lon'
    var_y = 'lat'
    var_element = 'ele'

# Read x, y, and elements from the grid file
x = vars[var_x][:]
y = vars[var_y][:]

elems = vars[var_element][:, :] - 1  # Move to 0-indexing by subtracting 1, elements indexing starts with '1' in netcdf file

if x is None or y is None or elems is None:
    print("*** ERROR *** No 'x (lon)', 'y (lat)', or 'element (ele)' data array given in file '%s'" % myfile)
    sys.exit(-1)

## 2.3 - Extracting and validating attribute data from the NetCDF file

A NetCDF file can contain multiple data arrays. In our case, we are interested in the data array `zeta_max` that contains the maximum water height for each node of the grid. 

We will read the `zeta_max` data array from our netCDF file utilizing the already parsed `vars`. If the data array is `None` or contains multiple time steps, the script prints an appropriate message and either exits the program or extract the first time step, respectively.

In [ ]:
# Read the 'attribute name' data array from the input file

attrname = "zeta_max"

data = vars[attrname][:]

if data is None:
    print("*** ERROR *** No '%s' data array given in file '%s'" % (attrname, infile))
    sys.exit(-1)

# Timesteps layer
if len(data.shape) > 1:
    print("*** The data file contains multiple time steps. The first time step (0) will be extracted.")
    data = data[0]

## Part 3: Map contouring and interactive plotting

Finally, we create an interactive plot using the `matplotlib` and `ipywidgets` libraries. The plot is based on a triangulation of data points, and the color of each triangle is determined by the corresponding data value.

You can use the sliders in the plot to interactively change the plot output by adjusting the input values. If you would like to learn more more about the options, please refer to the standalone Python script included in this tutorial or the README file.

In [ ]:

def interactive_plot(Max_Level, Intervals):
    # matplotlib: triangulation
    triang = tri.Triangulation(x, y, triangles=elems)
    # check if data array is masked
    try:
        if data.mask.any():
            # -99999 entries in 'data' array are usually masked, mask all corresponding triangles
            point_mask_indices = np.where(data.mask)
            tri_mask = np.any(np.in1d(elems, point_mask_indices).reshape(-1, 3), axis=1)
            triang.set_mask(tri_mask)

    except AttributeError:
        # the "mask" attribute was not found, assume that no -99999 values are in the dataset
        print("No 'mask' attribute found in file '%s'" % infile)

    levels = np.linspace(0, Max_Level, num=Intervals)
    # print("levels: %s\n" % levels)

    # Matplotlib: Color plot
    c = plt.tricontourf(triang, data, levels=levels, cmap=plt.cm.jet, extend='both')
    c.cmap.set_under("#000066")
    c.cmap.set_over("#880066")
    plt.colorbar(c, ticks=levels)
    plt.show()

# Create sliders for maxlevel and Intervals
maxlevel_slider = widgets.IntSlider(min=1, max=10, step=1, value=2)#max data value
Intervals_slider = widgets.IntSlider(min=2, max=30, step=1, value=16)#number of colors

# Use interact function to automatically create UI
interact(interactive_plot, Max_Level=maxlevel_slider, Intervals=Intervals_slider)
